# ASTR-79 Testing: Source Extraction (SEP)

This notebook tests the ASTR-79 source extraction implementation using the SEP-based extractor on synthetic difference images.

- SEPExtractor end-to-end on synthetic images
- Source list preview and basic stats (SNR, flux, size)
- Simple visualization overlays
- Aperture flux calculation check


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from uuid import uuid4

from src.domains.detection.extractors.sep_extractor import (
    SEPExtractor, SourceExtractionConfig
)

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'viridis'

print("✅ Imports successful for ASTR-79 testing")


In [ ]:
def create_synthetic_difference(size=(512, 512), n_sources=25, noise_sigma=5.0):
    image = np.random.normal(0.0, noise_sigma, size).astype(np.float64)
    rng = np.random.default_rng(42)
    yy, xx = np.indices(size)
    for _ in range(n_sources):
        x0 = int(rng.integers(30, size[1] - 30))
        y0 = int(rng.integers(30, size[0] - 30))
        amp = float(rng.uniform(50, 500))
        sigma = float(rng.uniform(1.5, 3.5))
        spot = amp * np.exp(-((xx - x0) ** 2 + (yy - y0) ** 2) / (2 * sigma**2))
        image += spot
    return image

# Generate synthetic difference image
D = create_synthetic_difference()
print(f"Synthetic difference image shape: {D.shape}, range: [{D.min():.2f}, {D.max():.2f}]")

plt.imshow(D)
plt.title('Synthetic Difference Image')
plt.colorbar()
plt.show()


In [ ]:
# Initialize SEP extractor
config = SourceExtractionConfig(
    detection_threshold=1.5,
    min_pixels=5,
    deblend_threshold=32.0,
    deblend_cont=0.005,
    clean_param=1.0,
    aperture_radius=4.0,
)
extractor = SEPExtractor(config)

# Extract sources
sources = extractor.extract_sources(D)
print(f"Detected {len(sources)} sources")

# Calculate aperture fluxes
ap_fluxes = extractor.calculate_source_flux(sources, D)

# Preview first 5 sources
for i, s in enumerate(sources[:5]):
    print(f"{i+1:02d}: (x,y)={s.coordinates}, flux={s.flux}, SNR={s.snr}, size={s.size}, ellip={s.ellipticity}")


In [ ]:
# Visualization: overlay detections
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(D, cmap='viridis')

# Draw circles at source locations
for s in sources:
    x, y = s.coordinates
    r = config.aperture_radius
    circle = plt.Circle((x, y), r, color='red', fill=False, linewidth=1.0)
    ax.add_patch(circle)

ax.set_title(f"Detections overlay (N={len(sources)})")
plt.colorbar(ax.images[0], ax=ax)
plt.show()


In [ ]:
# Refinement imports: analyzer, filters, scorer
from src.domains.detection.analyzers.photutils_analyzer import PhotutilsAnalyzer, PhotometryConfig
from src.domains.detection.filters.candidate_filter import CandidateFilter
from src.domains.detection.scorers.candidate_scorer import CandidateScorer, ScoringWeights
import sep
from scipy.ndimage import gaussian_filter

phot_analyzer = PhotutilsAnalyzer(PhotometryConfig(aperture_radius=4.0, annulus_inner=6.0, annulus_outer=9.0))
print("✅ Analyzer/Filter/Scorer ready")


In [ ]:
# Background/noise aided extraction with tighter parameters
config.detection_threshold = 3.0
config.min_pixels = 8
extractor = SEPExtractor(config)

D_smooth = gaussian_filter(D, sigma=1.0)
bkg = sep.Background(D_smooth.astype(np.float64))
D_sub = D_smooth - bkg.back()

sources_bkg = extractor.extract_sources_with_background(D_sub, bkg.back())
print(f"Detected with bkg: {len(sources_bkg)}")


In [ ]:
# Filtering and deduplication
filtered = CandidateFilter.filter_by_snr(sources_bkg, min_snr=3.5)
filtered = CandidateFilter.filter_by_size(filtered, min_size=10, max_size=600)
filtered = CandidateFilter.filter_by_shape(filtered, max_ellipticity=0.75)
H, W = D.shape
filtered = CandidateFilter.filter_by_position(filtered, {"x_min": 5, "y_min": 5, "x_max": W-5, "y_max": H-5})
filtered = CandidateFilter.filter_duplicates(filtered, min_distance=5.0)
print(f"After filtering: {len(filtered)}")


In [ ]:
# Photometry and morphology stats
phot = phot_analyzer.calculate_aperture_photometry(filtered, D_sub)
morph = phot_analyzer.measure_source_morphology(filtered, D_sub)
stats = phot_analyzer.calculate_source_statistics(filtered, D_sub)
print("Stats:", stats)
print("Photometry sample:", phot[:3])
print("Morphology sample:", morph[:3])


In [ ]:
# Scoring and ranking
scorer = CandidateScorer()
ranked = scorer.rank_candidates(filtered, D_sub, context={"width": W, "height": H, "edge_margin": 5})
print(f"After filtering: {len(filtered)}, top5 scores: {[round(s,3) for _, s in ranked[:5]]}")

top_k = 50
sel = [s for s, _ in ranked[:top_k]]

# Overlay
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(D, cmap='viridis')
for s in sel:
    x, y = s.coordinates
    circ = Circle((x, y), 5.0, color='lime', fill=False, linewidth=1.2)
    ax.add_patch(circ)
ax.set_title(f"Top-{top_k} ranked candidates")
plt.colorbar(ax.images[0], ax=ax)
plt.show()
